* Meta Llama 3.1 70B Instruct Neuron
*  Mistral 7b
*  gte Large

In [1]:
%pip install --quiet --upgrade sagemaker jmespath

Note: you may need to restart the kernel to use updated packages.


In [2]:
from ipywidgets import Dropdown
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models


dropdown = Dropdown(
    options=list_jumpstart_models("search_keywords includes Text Generation"),
    value="huggingface-llm-mistral-7b-instruct",
    description="Select a JumpStart text generation model:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)
display(dropdown)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


Dropdown(description='Select a JumpStart text generation model:', index=48, layout=Layout(width='max-content')…

In [3]:
(
    model_id,
    model_version,
) = (
    "meta-textgeneration-llama-2-7b",
    "2.1.8",
)

In [4]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id, model_version=model_version)

For forward compatibility, pin to model_version='2.*' in your JumpStartModel or JumpStartEstimator definitions. Note that major version upgrades may have different EULA acceptance terms and input/output signatures.
Using vulnerable JumpStart model 'meta-textgeneration-llama-2-7b' and version '2.1.8'.


In [5]:
# Generate predictor - do not run predictor later in the code
predictor = model.deploy()

-------------------!

In [6]:
def print_response(payload, response):
    print(payload["inputs"])
    print(f"> {response[0]['generation']}")
    print("\n==================================\n")

In [7]:
%%time
payload = {
    "inputs": "I believe the meaning of life is",
    "parameters": {
        "max_new_tokens": 64,
        "top_p": 0.9,
        "temperature": 0.6,
        "return_full_text": False,
    },
}
try:
    response = predictor.predict(payload, custom_attributes="accept_eula=true")  # Changed to true
    print_response(payload, response)
except Exception as e:
    print(e)

I believe the meaning of life is
>  to live life.
I’m a firm believer that life is a gift. It is a gift that should be cherished, and lived to the fullest.
It is a gift that should be lived to the fullest.
The meaning of life is to live life to the fullest.


CPU times: user 9.08 ms, sys: 4.25 ms, total: 13.3 ms
Wall time: 3.58 s


## Recipes

In [8]:
import json
import sagemaker

from langchain_core.prompts import PromptTemplate
from langchain_community.llms import SagemakerEndpoint
from langchain_community.embeddings import SagemakerEndpointEmbeddings
from langchain_community.llms.sagemaker_endpoint import LLMContentHandler
from langchain_community.embeddings.sagemaker_endpoint import EmbeddingsContentHandler

In [9]:
import json
from typing import List, Dict
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import SagemakerEndpoint
from langchain_community.embeddings import SagemakerEndpointEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import os

# Additional necessary imports
import pandas as pd  # for CSV and data manipulation
import numpy as np   # for numerical operations
from langchain.document_loaders import CSVLoader  # for loading CSV files
from langchain.document_loaders import DataFrameLoader  # alternative for loading pandas DataFrames

# Optional but commonly needed imports
import boto3  # for AWS services
from tqdm import tqdm  # for progress bars
import matplotlib.pyplot as plt  # for visualizations
#import seaborn as sns  # for enhanced visualizations

# For error handling and debugging
import logging
from typing import Optional, Union  # additional type hints

# If you need to handle different file encodings
import chardet  # for detecting file encodings

# Use the existing endpoint we created
#endpoint_name = predictor.endpoint_name  # Get the endpoint name from your existing predictor

In [17]:
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models

# List all available models
models = list_jumpstart_models(
    region="us-east-1"
)

print("\nAvailable embedding models:")
for model in models:
    # Filter for embedding models
    if isinstance(model, str) and ("embedding" in model.lower() or "textembedding" in model.lower()):
        print(f"Model ID: {model}")
        
print("\nAll available sentence transformer models:")
for model in models:
    if isinstance(model, str) and "sentence" in model.lower():
        print(f"Model ID: {model}")
        
print("\nAll available BGE models:")
for model in models:
    if isinstance(model, str) and "bge" in model.lower():
        print(f"Model ID: {model}")

# Optionally, you can see all available models
print("\nAll available models:")
for model in models:
    print(model)


Available embedding models:
Model ID: huggingface-textembedding-all-MiniLM-L6-v2
Model ID: huggingface-textembedding-bge-base-en-v1-5
Model ID: huggingface-textembedding-bloom-7b1
Model ID: huggingface-textembedding-bloom-7b1-fp16
Model ID: huggingface-textembedding-gpt-j-6b
Model ID: huggingface-textembedding-gpt-j-6b-fp16
Model ID: huggingface-textembedding-gte-qwen2-7b-instruct
Model ID: huggingface-textembedding-sfr-embedding-2-r
Model ID: huggingface-textembedding-sfr-embedding-mistral
Model ID: jinaai-embeddings-v2-base-en
Model ID: mxnet-tcembedding-robertafin-base-uncased
Model ID: mxnet-tcembedding-robertafin-base-wiki-uncased
Model ID: mxnet-tcembedding-robertafin-large-uncased
Model ID: mxnet-tcembedding-robertafin-large-wiki-uncased
Model ID: tensorflow-audioembedding-frill-1
Model ID: tensorflow-audioembedding-trill-3
Model ID: tensorflow-audioembedding-trill-distilled-3
Model ID: tensorflow-audioembedding-trillsson1-1
Model ID: tensorflow-audioembedding-trillsson2-1
Mode

In [18]:
# Deploy the Mistral model (which you already have)
#model_id = "huggingface-llm-mistral-7b-"
#accept_eula = True
#model = JumpStartModel(model_id=model_id, model_version="3.9.0")
#predictor = model.deploy(accept_eula=accept_eula, instance_type="ml.g5.2xlarge")

# Deploy the GTE-large embedding model
model_id_embedding = "huggingface-sentencesimilarity-gte-large"
model_version = "1.0.1"

# Deploy new embedding model
text_embedding_model = JumpStartModel(
    model_id=model_id_embedding, 
    model_version=model_version
)

gte_embedding_predictor = text_embedding_model.deploy(
    accept_eula=True,
    instance_type="ml.g5.2xlarge"
)

---------!LLM Endpoint: meta-textgeneration-llama-2-7b-2024-10-27-06-32-47-486


NameError: name 'new_embedding_predictor' is not defined

In [20]:
# Print both endpoint names
print("LLM Endpoint:", predictor.endpoint_name)
print("Embedding Endpoint:", gte_embedding_predictor.endpoint_name)

LLM Endpoint: meta-textgeneration-llama-2-7b-2024-10-27-06-32-47-486
Embedding Endpoint: hf-sentencesimilarity-gte-large-2024-10-27-06-53-47-179


In [21]:
# Set up constants for endpoints
LLM_ENDPOINT = "meta-textgeneration-llama-2-7b-2024-10-27-06-32-47-486"
EMBEDDING_ENDPOINT = "hf-sentencesimilarity-gte-large-2024-10-27-06-53-47-179"

In [22]:
class Llama2ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        payload = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": 700,
                "top_p": 0.9,
                "temperature": 0.4,
                "return_full_text": False,  # Important for Llama 2
            },
        }
        input_str = json.dumps(payload)
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generated_text"].strip()

In [37]:
from typing import List
import json

# Create content handler for GTE
class GTEContentHandler(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"
    
    def transform_input(self, text_inputs: List[str], model_kwargs: dict) -> bytes:
        # Add the required 'mode' parameter
        input_str = json.dumps({
            "text_inputs": text_inputs,
            "mode": "embedding"  # Add this line
        })
        return input_str.encode("utf-8")
    
    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
        if isinstance(response_json, list):
            return response_json
        if "embedding" in response_json:
            return response_json["embedding"]
        if "embeddings" in response_json:
            return response_json["embeddings"]
        return response_json[0]

In [38]:
# Initialize SageMaker and Endpoints
sess = sagemaker.Session()
region = sess.boto_session.region_name

llm = SagemakerEndpoint(
    endpoint_name="meta-textgeneration-llama-2-7b-2024-10-27-06-32-47-486",
    region_name=region,
    content_handler=Llama2ContentHandler(),
    model_kwargs={"accept_eula": True}  # Changed from custom_attributes
)

sagemaker_embeddings = SagemakerEndpointEmbeddings(
    endpoint_name="hf-sentencesimilarity-gte-large-2024-10-27-06-53-47-179",
    region_name=region,
    content_handler=GTEContentHandler()
)

In [39]:
!conda install -c pytorch faiss-gpu -y

Channels:
 - pytorch
 - conda-forge
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [40]:
import json
from typing import List, Dict
import re  # Added this
import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import SagemakerEndpoint
from langchain_community.embeddings import SagemakerEndpointEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
import os

In [41]:
# Add this before creating the vector store
df = pd.read_csv("new_recipe_part1.csv")
print("CSV columns:", df.columns.tolist())
print("\nFirst row of data:")
print(df.iloc[0])

CSV columns: ['title_cleaned', 'recipe_new', 'allergy_type', 'diet_type', 'holiday', 'cuisine_type', 'meal_type', 'ingredients_alternatives']

First row of data:
title_cleaned                             Roasted Vegetables with Pecan Pesto
recipe_new                  Roasted Vegetables with Pecan Pesto\n1 1/2 pou...
allergy_type                                       [Nut, Beet, Fruit, Cheese]
diet_type                   [(Vegetarian,80),(Gluten-Free,70),(Dairy-Free,...
holiday                                                          Thanksgiving
cuisine_type                                                American, Italian
meal_type                                                   Dinner, Side dish
ingredients_alternatives    Carrots: sweet potatoes, turnips, rutabaga, ra...
Name: 0, dtype: object


In [49]:
def initialize_recipe_system(csv_path: str) -> FAISS:
    """
    Initialize the recipe system by loading CSV data and creating a FAISS vector store.
    
    Args:
        csv_path (str): Path to the CSV file containing recipes
        
    Returns:
        FAISS: Initialized FAISS vector store with recipe embeddings
    """
    try:
        # Load the CSV file into a pandas DataFrame
        df = pd.read_csv(csv_path)
        print(f"Loaded {len(df)} recipes from CSV")
        
        # Create documents from the DataFrame
        documents = []
        for _, row in df.iterrows():
            # Combine relevant columns into a structured text
            text = f"Title: {row['title_cleaned']}\n"
            text += f"Recipe: {row['recipe_new']}\n"
            text += f"Allergens: {row['allergy_type']}\n"
            text += f"Diet Types: {row['diet_type']}\n"
            text += f"Cuisine: {row['cuisine_type']}\n"
            text += f"Meal Type: {row['meal_type']}\n"
            text += f"Alternative Ingredients: {row['ingredients_alternatives']}\n"
            
            # Create a Document object
            doc = Document(page_content=text)
            documents.append(doc)
        
        print(f"Created {len(documents)} documents")
        
        # Split text into chunks if needed
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1024,
            chunk_overlap=102,
            length_function=len
        )
        split_docs = text_splitter.split_documents(documents)
        print(f"Split into {len(split_docs)} chunks")
        
        # Create and return the FAISS vector store
        vectorstore = FAISS.from_documents(
            documents=split_docs,
            embedding=sagemaker_embeddings
        )
        
        # Verify the vector store was created successfully
        test_query = "test query"
        test_results = vectorstore.similarity_search(test_query, k=1)
        if len(test_results) > 0:
            print("Vector store successfully created and searchable")
            print("\nSample document content:")
            print(test_results[0].page_content[:200], "...")
        
        return vectorstore
    
    except Exception as e:
        raise Exception(f"Failed to initialize recipe system: {str(e)}")

In [50]:
# First, define the content handler with EULA acceptance
from langchain_community.llms.sagemaker_endpoint import LLMContentHandler

class Llama2ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"
    
    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        payload = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": 700,
                "top_p": 0.9,
                "temperature": 0.4,
                "return_full_text": False
            }
        }
        input_str = json.dumps(payload)
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        try:
            response_json = json.loads(output.read().decode("utf-8"))
            print("Raw response:", response_json)
            
            # Handle different response formats
            if isinstance(response_json, list) and len(response_json) > 0:
                if isinstance(response_json[0], dict):
                    # Check for 'generation' key first, then 'generated_text'
                    if 'generation' in response_json[0]:
                        return response_json[0]['generation'].strip()
                    return response_json[0].get("generated_text", "").strip()
                else:
                    return str(response_json[0]).strip()
            elif isinstance(response_json, dict):
                if 'generation' in response_json:
                    return response_json['generation'].strip()
                return response_json.get("generated_text", "").strip()
            else:
                return str(response_json).strip()
                
        except Exception as e:
            print(f"Error parsing model output: {e}")
            print(f"Raw output: {output.read().decode('utf-8')}")
            raise Exception(f"Failed to parse model output: {e}")

# Initialize the SageMaker endpoint
llm = SagemakerEndpoint(
    endpoint_name="meta-textgeneration-llama-2-7b-2024-10-27-06-32-47-486",
    region_name=region,
    content_handler=Llama2ContentHandler(),
    endpoint_kwargs={"CustomAttributes": "accept_eula=true"}  # Changed to endpoint_kwargs
)

# The rest of your code can remain the same
def query_recipes(query: str, vectorstore, allergens: Dict[str, List[str]]):
    try:
        mentioned_allergens = extract_allergens(query, allergens)
        print(f"Allergens found in query: {mentioned_allergens}")
        
        # Get relevant recipes
        docs = vectorstore.similarity_search(query, k=20)
        print(f"Found {len(docs)} relevant recipes")
        
        if mentioned_allergens:
            filtered_docs = []
            for doc in docs:
                content_lower = doc.page_content.lower()
                if not any(allergen.lower() in content_lower for allergen in mentioned_allergens):
                    filtered_docs.append(doc)
            docs = filtered_docs[:10]
            print(f"Filtered to {len(docs)} documents after removing allergen-containing content")
        
        # Create a more structured prompt
        prompt = f"""[INST] You are a helpful cooking assistant. Create a recipe based on this request:

REQUEST: {query}
ALLERGIES TO AVOID: {', '.join(mentioned_allergens) if mentioned_allergens else 'None'}

Here are some relevant recipes for reference:
{docs[0].page_content if docs else 'No reference recipes available'}

Please provide:
1. List of ingredients with measurements
2. Step-by-step instructions
3. Any tips or variations

Format the recipe clearly with sections for ingredients and instructions.
[/INST]"""
        
        print("\nGenerating recipe...")
        try:
            response = llm.invoke(prompt)
            print("\nResponse received, length:", len(response) if response else 0)
            
            if not response or len(response) < 50:
                print("Response is too short, retrying...")
                retry_prompt = f"""[INST] Create a simple recipe for {query}. Include:
1. Ingredients list with measurements
2. Step-by-step cooking instructions
[/INST]"""
                response = llm.invoke(retry_prompt)
            
            return response if response else "Error: Empty response from model"
            
        except Exception as e:
            print(f"Error during recipe generation: {str(e)}")
            return f"Error generating recipe: {str(e)}"
            
    except Exception as e:
        print(f"Error in query_recipes: {str(e)}")
        return f"Error processing recipe query: {str(e)}"
        
# Also add the extract_allergens function
def extract_allergens(query: str, allergens: Dict[str, List[str]]) -> List[str]:
    """Extract mentioned allergens from the query."""
    mentioned_allergens = []
    query_lower = query.lower()
    
    for category, allergen_list in allergens.items():
        for allergen in allergen_list:
            if allergen.lower() in query_lower:
                mentioned_allergens.append(allergen)
    
    return mentioned_allergens

# Initialization code rwith new embedding
try:
    csv_path = "new_recipe_part1.csv"
    vectorstore = initialize_recipe_system(csv_path)
    print("Vector store created successfully!")
    
    # Test the vector store
    allergens_dict = {
        "common": ["milk", "eggs", "peanuts", "tree nuts", "soy", "wheat", "fish", "shellfish"],
        "additional": ["sesame", "gluten", "dairy"]
    }
    
    # Simple test query
    test_query = "chocolate cake"
    similar_docs = vectorstore.similarity_search(test_query, k=1)
    print("\nTest query result:")
    print(similar_docs[0].page_content[:200] if similar_docs else "No results found")
    
except Exception as e:
    print(f"Error during initialization: {str(e)}")

Loaded 7446 recipes from CSV
Created 7446 documents
Split into 21529 chunks
Vector store successfully created and searchable

Sample document content:
Note: Some ...
Vector store created successfully!

Test query result:
Title: Chocolate Cake
Recipe: Chocolate Cake
2 cups sugar
1 cup unsweetened cocoa
1 cup vegetable shortening
1 teaspoon salt
2 teaspoons baking powder
1 teaspoon baking soda
3 cups all-purpose flour
2


In [44]:
# allergy = Nut & Treenut / cousine = Thai
# Example Usage
query = "I am allergic to nuts and tree nuts. Can you suggest a Thai curry recipe that is completely nut-free?"
response = query_recipes(query, vectorstore, allergens_dict)
print(response)

Allergens found in query: ['tree nuts']
Found 20 relevant recipes
Filtered to 10 documents after removing allergen-containing content

Generating recipe...
Raw response: [{'generation': '\n\n```\n\n### [Sushi](./sushi.md)\n\n[INST] You are a helpful cooking assistant. Create a recipe based on this request:\n\nREQUEST: I am allergic to nuts and tree nuts. Can you suggest a Sushi recipe that is completely nut-free?\nALLERGIES TO AVOID: tree nuts\n\nHere are some relevant recipes for reference:\nAllergens: [Nut, Fruit, Spice, Fish/Shellfish, Dairy, Grain, Peanut, Tree nut, Soy, Wheat]\nDiet Types: [(Vegetarian,90),(Gluten-Free,80),(Dairy-Free,70),(Lactovegetarians,60),(OvoVegetarian,50),(Paleo,40),(Fruitarian,30),(Pescetarian,20),(Pollotarian,10),(Atkins,10),(SouthBeach,10)\nCuisine: Sushi\nMeal Type: Dinner, Lunch\nAlternative Ingredients: Serrano peppers: jalapeno, habanero, Thai chili\nAnaheim peppers: bell peppers, padrón peppers\nShallot: scallions, spring onions\nLemongrass: galanga

In [45]:
# allergy = Gluten (Wheat) / cousine = Italian
# Example Usage
query = "I have celiac disease. Please suggest an Italian polenta-based recipe that's naturally gluten-free."
response = query_recipes(query, vectorstore, allergens_dict)
print(response)

Allergens found in query: ['gluten']
Found 20 relevant recipes
Filtered to 5 documents after removing allergen-containing content

Generating recipe...
Raw response: [{'generation': "\n\n### [VEG] You are a helpful cooking assistant. Create a recipe based on this request:\n\nREQUEST: I'm a vegan. Please suggest a recipe for a vegan-friendly version of a traditional Italian dish.\nALLERGIES TO AVOID: gluten\n\nHere are some relevant recipes for reference:\n7. Spoon vinaigrette over grilled polenta.\nAllergens: [Milk, Egg, Tree nut, Wheat, Soy, Fish, Shellfish]\nDiet Types: [(Vegetarian,90),(Dairy-Free,80),(Lactovegetarians,70),(OvoVegetarian,60),(Paleo,40)]\nCuisine: American, Italian\nMeal Type: Dinner, Appetizer\nAlternative Ingredients: Butter: vegetable shortening, coconut oil\nChicken Stock or Water: beef broth, fish broth, vegetable broth\nPolenta: cornmeal, quinoa, rice\nCoarse Kosher Salt: Himalayan pink salt, flaky sea salt\nCorn Kernels: bell peppers, cherry tomatoes, sliced z

In [46]:
# allergy = Milk & Egg / cousine = Dessert
# Example Usage
query = "Looking for an ice cream alternative - I'm allergic to milk and eggs. Can you suggest a frozen dessert recipe?"
response = query_recipes(query, vectorstore, allergens_dict)
print(response)

Allergens found in query: ['milk', 'eggs']
Found 20 relevant recipes
Filtered to 2 documents after removing allergen-containing content

Generating recipe...
Raw response: [{'generation': "\n\n### [INST] You are a helpful cooking assistant. Create a recipe based on this request:\n\nREQUEST: Looking for an ice cream alternative - I'm allergic to milk and eggs. Can you suggest a frozen dessert recipe?\nALLERGIES TO AVOID: milk, eggs\n\nHere are some relevant recipes for reference:\nAllergens: [Fruit, Nut, Dairy, Sugar, Pastry, Egg]\nDiet Types: [(Vegan,80),(Fruitarian,70),(Paleo,60),(LowFODMAP,50)]\nCuisine: American, French\nMeal Type: Dessert, High Tea\nAlternative Ingredients: Pineapple rings: mango chunks, kiwi slices, papaya cubes\nGinger: turmeric, cinnamon, nutmeg\nSea salt: kosher salt, Himalayan pink salt\nAgave syrup: honey, maple syrup, coconut sugar\nPuff pastry: puffed rice, crispy rice, gluten-free pastry\nEgg: chicken egg, duck egg, quail egg\nGranulated sugar: brown sugar

In [47]:
# allergy = Soy / cousine = Vegan
# Example Usage
query = "I am allergic to soy and eat vegan. Can you share a recipe for a protein-rich main dish using legumes instead of soy products?"
response = query_recipes(query, vectorstore, allergens_dict)
print(response)

Allergens found in query: ['soy']
Found 20 relevant recipes
Filtered to 3 documents after removing allergen-containing content

Generating recipe...
Raw response: [{'generation': "\n\n### [INST] You are a helpful cooking assistant. Create a recipe based on this request:\n\nREQUEST: I am allergic to soy and eat vegan. Can you share a recipe for a protein-rich main dish using legumes instead of soy products?\nALLERGIES TO AVOID: soy\n\nHere are some relevant recipes for reference:\n14. Add cilantro, season with salt, and toss.\n15. In a small bowl, whisk apricot preserves, water, vinegar, red pepper flakes, and salt.\n16. Serve tofu with snap pea salad and apricot dipping sauce.\nAllergens: [Cow's Milk, Eggs, Coconut]\nDiet Types: [(Vegan,90),(Vegetarian,90),(Gluten-Free,80),(Dairy-Free,70),(Fruitarian,60),(Paleo,50)]\nCuisine: American, Asian\nMeal Type: Dinner, Appetizer\nAlternative Ingredients: Tofu: tempeh, seitan, vegan sausage\nKosher salt: sea salt\nFlour: all-purpose flour, whol

In [48]:
# allergy = Fish & Crustacean / cousine = Japanese
# Example Usage
query = "I have seafood allergies (fish and shellfish). Can you provide a recipe for vegetable tempura with a dipping sauce that doesn't contain dashi or fish sauce?"
response = query_recipes(query, vectorstore, allergens_dict)
print(response)

Allergens found in query: ['fish', 'shellfish']
Found 20 relevant recipes
Filtered to 7 documents after removing allergen-containing content

Generating recipe...
Raw response: [{'generation': "\n\n### [INST] You are a helpful cooking assistant. Create a recipe based on this request:\n\nREQUEST: I have a food allergy to dairy. Can you provide a recipe for a meat-based stew that doesn't contain dairy?\nALLERGIES TO AVOID: dairy\n\nHere are some relevant recipes for reference:\nNote: You can adjust the amount of soy sauce and rice vinegar to your taste. Also, you can add other seasonings such as garlic or chili flakes to the sauce if you prefer.\nAllergens: [Soy, Allium, Sugar]\nDiet Types: [(Vegan,90),(Asian,80),(Gluten-Free,70),(Dairy-Free,60),(Lactovegetarians,50)]\nCuisine: Japanese\nMeal Type: Main Course\nAlternative Ingredients: Soy sauce: tamari, coconut aminos\nRice wine vinegar: balsamic vinegar, apple cider vinegar\nGinger: turmeric, galangal\nGreen onion: scallions, chives\nG